In [1]:
import json, base64
s = {
    "fileNames": ["train.csv", "test.csv"],
    "userId": "servtech.ai_julie",
    "groupId": "servtech.ai",
    "projectId": "p00226",
    "job_id": "20230814082335099"
}
s = json.dumps(s)

b = s.encode("UTF-8")
e = base64.b64encode(b)
argv = e.decode("UTF-8")

print(f'python run.py "{argv}"')

python run.py "eyJmaWxlTmFtZXMiOiBbInRyYWluLmNzdiIsICJ0ZXN0LmNzdiJdLCAidXNlcklkIjogInNlcnZ0ZWNoLmFpX2p1bGllIiwgImdyb3VwSWQiOiAic2VydnRlY2guYWkiLCAicHJvamVjdElkIjogInAwMDIyNiIsICJqb2JfaWQiOiAiMjAyMzA4MTQwODIzMzUwOTkifQ=="
